# Design Considerations

## Part 1: Identify FIVE design decisions for this system.
Argue for how this design decision is significant to the system, and your plans to analyze this design. If needed, use the dataset provided to you.

### 1. Sentences per Chunk
One important factor to consider is the number of sentences per chunk, which is used by the `DocumentProcessing` class to split the corpus documents into smaller chunks before the pipeline calculates the embedding vector for each chunk. During the retrieval and generation process, these chunks will be used as context and provided as input to the generator LLM that generates the answer to the question. Therefore, the number of sentences in these chunks will directly affect how much contextual information the generator will have available to generate a response. A high number of sentences per chunk provides the generator with lots of contextual information, which might be useful for complex questions. However, too much information may result in irrelevant information being provided to the generator, which may confuse the model and produce unreliable answers. A small number of sentences per chunk could decrease the chances of providing irrelevant information, but could also decrease the chances of including the true answer in the context.

**Analysis Plan**
To analyze the effects of different numbers of sentences per chunk, a validation set of questions, like the ones in [qa_resources/questions.csv](../qa_resources/questions.csv), could be used to test how the RAG system performs when the corpus documents have been processed using different numbers of sentences per chunk. For example we could use sentence per chunk values of [1,2,3,4,5,6] and for each value, we can calculate the document embeddings using the specified sentences per chunk, and use those embeddings to retrieve the context to be used to generate an answer for each question in the validation set. The answers to each question can be evaluated using automated metrics such as exact match and transformer match metrics. Additionally, the time it takes to get the context and generate an answer for a question can be tracked to identify any potential differences in latency for different sentences per chunk values. The sentences per chunk value that results in the best balance of high transformer match and low latency would be chosen for deployment to the system.

### 2. Similarity Measure
Another important design consideration is the similarity measure used in the Nearest Neighbor search when calculating the distance between two embedding vectors during the context retrieval process. This is important because it can affect how relevant the retrieved context is to a given question. The distance metric must ensure that higher distances are associated with dissimilar text and lower distances are associated with similar text strings. Different distance metrics can lead to different results for the same points, so it's important to carefully choose the distance measure that results in the best system performance. Some popular distance metrics include Manhattan distance (aka "city block" distance), Euclidean distance (aka "straight line" distance), Minkowski distance (a generalized version of Manhattan and Euclidean distances), and cosine similarity which measures the angle between two vectors.

**Analysis Plan**

To analyze the different similarity measures, each variation (Manhattan distance, Euclidean distance, Minkwoski distance with different values for the p parameter, and cosine similarity) can be used to retrieve context used to generate answers for validation set of questions with known answers. The exact match and transformer match metrics can then be calculated for each run, so that the quality of the system outputs can be compared for the context search using each similarity measure. Additionally, the mean search time per question can be tracked so we can identify any differences in efficiency between the similarity measures. The similarity measure resulting in the best balance of transformer match and efficiency would be chosen for deployment. Note that for the analysis, the transformer match is calculated using the output of the generator that creates a response using the context identified using the KNN search. This does not directly evaluate the outputs of the KNN search, but additional analysis couold be done to evaluate the nearest neighbors directly. This could be done by calculating the quality of the ranked nearest samples by finding the mean avereage precision, where relevant matches are determined by the transformer match matric applied to the nearest neighbor chunk and the ground truth answer. This analysis would more directly determine how the similarity measure affects the relevance of the retrieved context chunks.

### 3. Number of Nearest Neighbors (K)
Another important design consideration is the number of nearest neighbors (k value) to be used in the search service when retrieving relevant context chunks for a given question. Lower k values may be more sensitive to noisy data (overlap between chunks of corpus documents in our case), but can also be useful to account for complex decision boundaries between clusters of document chunks. Higher k values produce results that are less affected by outliers or overlap in document chunk clusters, but can introduce bias from irrelevant sentences that may become included when we expand k. Similar to the sentences per chunk analysis, a small k value could decrease the chances of providing irrelevant information, but could also decrease the chances of including the true answer in the context. Ideally, k should be large enough to capture chunks containing the true answer to the question, but small enough to exclude irrelevant information as much as possible.

**Analysis Plan**

To find the best k value, the search service can be run on a validation set of questions, using a different k value for each run. For each run, for each question, the k retrieved context chunks will then be used as input with the question to the generater that will create an answer to the question. The answers can be evaluated using the exact match and transformer match metrics for each run, so that the quality of the answers can be compared for each k value. Additionally, mean search time per question can be tracked so we can identify any differences in efficiency for different k values. The k value resulting in the best balance of transformer match metric and search time would be chosen for deployment.

### 4. Embedding Model Selection
Another important design consideration is the embedding model used to generate text embedding vectors from the corpus documents and input questions. The `all-MiniLM-L6-v2` sentence transformer model was provided in the case study as an example, with additional pretrained sentence transformer models described at [https://sbert.net/docs/sentence_transformer/pretrained_models.html](https://sbert.net/docs/sentence_transformer/pretrained_models.html). Each model produces different embeddings, leading to differences in cluster separability of text chunks and documents, leading to differing KNN search results, resulting in different contextual information and therefore generated responses. Models that produce embeddings that put text chunks of the same topic (i.e. from the same document) closer together, and text of different topics further apart (i.e. models that produce well-defined clusters of text) will ultimately lead to the most relevant context retrieved and best generated answer results. Additionally, the different embedding models all have different sizes, speeds, and accuracies. Typically, the larger models perform better but may be slower. Smaller models can be quicker and faster, but may perform worse. Furthermore, the models produce embedding vectors of different sizes, which is important to consider since it will affect our system's storage requirements since we are saving the embedding vectors for each document chunk. Additionally, smaller embedding vectors are more computationally efficient, but contain less information and therefore may not fully represent key aspects that differentiate each text chunk. Larger embedding vectors are more computationally intensive to search, but contain more information and may represent more complex aspects that differentiate text. However, larger embedding vectors may also contain more extraneous, or noisy information unrelated to the text topic such as specific punctuation or phrasing information. Ideally, the embedding vectors should be large enough to capture key aspects of the text that are significant for differentiating between ideas, but small enough so they don't contain too much noisy information that isn't important for a sentence (e.g. punctuation or paraphrasing).

**Analysis Plan**

To analyze and compare each model, embeddings can be computed for the corpus documents using a few different models such as `all-MiniLM-L6-v2`, `all-mpnet-base-v2`, `all-MiniLM-L12-v2`, and `paraphrase-MiniLM-L3-v2`. The clustering quality of the resulting embeddings can be measured using silhouette score. A higher silhouette score indicates better intra-cluster cohesion and inter-cluster separation. Therefore, a higher silhouette score indicates that using the model in our system will result in better clustering of document text leading to more relevant context retrieval and ideally better generated answers. To further support this, a validation set of known questions can be run through our RAG system using the embeddings produced by each of the 4 models, and the transformer match metric can be calculated for the generated answer produced using the context retrieved from the KNN search using the embedding vectors produced by each model. Additionally, response time per question can be measured to analyze any potential differences in efficiency of using the embeddings produced by each model. The model that leads to the highest transformer match value with an efficient response time is the one that should be chosen for deployment.

### 5. Generator Model Selection
Another important design consideration is the generator model used to produce an answer to a given question using the provided context.

**Analysis Plan**


### References

## Part 2: Analyze TWO out of the five design decisions. Add the results of your analysis.

#### Import necessary libraries

In [ ]:
import sys
sys.path.insert(1,'../')

### 1. TBD

### 2. TBD